# 2A2E: Intro to Computer Engineering 2 - Solutions [Shippet to test Jupyter notebook]

The following is a minimal snippet to test your Jupyter notebook on SageMaker Studio. We will reuse this approach for C18.

In [26]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [27]:
# Testing infrastructure copied from lecture notes
# 

from random import randint, random
from copy import deepcopy


def run_one_test(args, result, func):
    """
    Test if the input function generates the given result with the input args.
    Args:
        args: A tuple of arguments to apply to the function
        result: The (single) result to expect from the function
        func: The function to test
    Returns:
        True if the function produces an output that equals result
    """    
    # apply func to args
    # *args expands the input tuple and uses the results as arguments to the function    
    if func(*args) == result:        
        return True
    else:
        return False

def run_tests(values, func):
    """
    Test the input function with multiple input/ouput value pairs
    Args:
        values: a list of input-output pairs 
        func: The function to test
    """
    successes = 0
    failures = 0
    for test_values in deepcopy(values):
        if run_one_test(test_values[0], test_values[1], func):
            successes += 1
        else:
            failures += 1
    print('{} successes and {} failures'.format(successes, failures))



def time_tests(tests, func, repeats_per_test=100):
    """
    Time a collection of tests on the given function. This assumes that len(tests[i][0][0]) gives the problem size.

    Note that all timing results include the cost of one deepcopy per test on the arguments. 

    Args:
        tests: A list of input/output pairs that can be sent to run_one_test. 
        func: The function to test.
        
    Returns:
        problem_sizes: the sizes of the problems tested
        problem_means: a mean runtime in seconds for each problem size
        problem_stdevs: a standard deviation for each mean
    """
    
    # for efficiency when we copy we are not going to use the outcomes 
    # so create a new list of test without outcomes
    test_list = [test[0] for test in tests]
    
    timing_results = dict()
    
    for r in range(repeats_per_test):
        # do one big copy of all tests, rather than copying each test individually
        for args in deepcopy(test_list):
            # run the test
            res = %timeit -r 1 -n 1 -o -q func(*args)
            # get the input size we tests
            n = len(args[0])
            # store the result in the dictionary
            # each n maps to a list of two lists, one for mean, one for stddev
            timing_results.setdefault(n, []).append(res.average)

    # the individual problem sizes are the keys from the dictionary
    problem_sizes = np.array(list(timing_results.keys()))
    # calc means for the problem, 
    problem_means = np.array([np.mean(timing_results[n]) for n in problem_sizes])
    # calc stdevs for the problem
    problem_stdevs = np.array([np.std(timing_results[n]) for n in problem_sizes])
    return problem_sizes, problem_means, problem_stdevs

    


## Question 1: Hotter/Colder

The goal of your algorithm is to guess a secret integer between $1$ and $n$. For each guess your programme should either terminate in the case that you have guessed the correct answer, or it will recieve *hotter* ($1$) if this guess is closer than the previous guess, or *colder* ($-1$) if the answer is further away than the previous answer. Should your programme guess the same number twice you will recieve a tepid response ($0$). The secret integer is always in the range $[1...n]$ (i.e. you do not have to consider the case that the secret does not exist).


In [35]:

# Define some constants to use in the guessing game

HOTTER="hotter"
COLDER="colder"
TEPID="tepid"
CORRECT="correct"

def guess_secret(secret, current_guess, previous_guess):
    """
    This function catpures the rules of the guessing game.
    Args:
        secret: The secret to be guessed
        current_guess: The guess you're making
        previous_guess: The guess you made before the current one
    """
    if secret == current_guess:
        return CORRECT
    elif current_guess == previous_guess:
        return TEPID
    else:
        # how far was previous guess away
        previous_diff = abs(secret - previous_guess)
        # how far away is the current guess
        current_diff = abs(secret - current_guess)
        if current_diff < previous_diff:
            return HOTTER
        else:
            return COLDER

In [36]:
test_n = 100
test_secret = 30
print(guess_secret(test_secret, 0, 0))
print(guess_secret(test_secret, 20, 0))
print(guess_secret(test_secret, 25, 20))
print(guess_secret(test_secret, 55, 25))
print(guess_secret(test_secret, test_secret, 55))


tepid
hotter
hotter
colder
correct
